In [1]:
import polars as pl
import duckdb


In [2]:
def calculate_moving_averages(data, short_window, long_window):
    data = data.with_columns(
        data['close'].rolling_mean(window_size=int(short_window)).alias('Short_MA'),
        data['close'].rolling_mean(window_size=int(long_window)).alias('Long_MA')
    )
    return data


In [3]:
def simulate_trading(data):
    cash = 100000
    shares_held = 0
    simulation_results = []

    normalization_factor = cash / data['close'][0]

    # Create conditions for buying and selling
    buy_condition = (data['Short_MA'] > data['Long_MA']) & (data['Short_MA'].shift(1) <= data['Long_MA'].shift(1))
    sell_condition = (data['Short_MA'] < data['Long_MA']) & (data['Short_MA'].shift(1) >= data['Long_MA'].shift(1))

    for i in range(data.height):
        asset_value = cash + shares_held * data['close'][i]
        
        if buy_condition[i]:
            shares_to_buy = cash // data['close'][i]
            cash -= shares_to_buy * data['close'][i]
            shares_held += shares_to_buy
        elif sell_condition[i]:
            cash += shares_held * data['close'][i]
            shares_held = 0

        simulation_results.append({
            'Cash': cash,
            'Shares Held': shares_held,
            'Asset Value': cash + shares_held * data['close'][i],
            'Buy Point': buy_condition[i],
            'Sell Point': sell_condition[i],
            'Normalized Stock Price': data['close'][i] * normalization_factor
        })

    simulation_df = pl.DataFrame(simulation_results)
    return simulation_df


In [14]:
def test_strategy(args, data):
    short_window, long_window = args
    data = calculate_moving_averages(data.clone(), short_window, long_window)  # Clone to avoid modifying the original data
    simulation_df = simulate_trading(data)
    return simulation_df


In [51]:
from concurrent.futures import ProcessPoolExecutor
import polars as pl
import itertools

# Function to compute final asset value for a combination
def compute_final_asset_value(combination, data_df):
        short_window, long_window = combination
        return test_strategy((short_window, long_window), data_df)['Asset Value'][-1]  # Return the final asset value


def sensitivity_analysis(data_df, short_range, long_range):
    # Prepare the combinations of short and long windows
    combinations = list(itertools.product(short_range, long_range))

    
    # Use ThreadPoolExecutor to parallelize the computation
    with ProcessPoolExecutor() as executor:
        final_asset_values = list(executor.map(compute_final_asset_value, combinations,  [data_df]*len(combinations)))

    # Create the results DataFrame
    results = [{'Short_Window': short, 'Long_Window': long, 'Final_Asset_Value': value} 
               for ((short, long), value) in zip(combinations, final_asset_values)]
    results_df = pl.DataFrame(results)
    
    # Reshape the DataFrame
    results_df = results_df.pivot(index='Short_Window', columns='Long_Window', values='Final_Asset_Value')

    return results_df


In [52]:
import plotly.express as px

def analyze_and_visualize(df):
    # Convert Polars DataFrame to Pandas for easier handling with Plotly
    pd_df = df.to_pandas().set_index('Short_Window')

    # Finding the Highest Value and its Indices
    highest_value = pd_df.values.max()
    index_of_highest_value = pd_df.stack().idxmax()  # Returns a tuple of indices (row, column)

    # Calculating the Mean
    mean_value = pd_df.values.mean()

    # Calculating the Median
    median_value = pd_df.median().median()

    # Calculating the Percentage Difference
    percentage_difference = ((highest_value - mean_value) / mean_value) * 100

    # Output the results
    print(f'Highest Value: {highest_value} at {index_of_highest_value}')
    print(f'Mean Value: {mean_value}')
    print(f'Median Value: {median_value}')
    print(f'Percentage Difference: {percentage_difference:.2f}%')

    # Visualizing the Data with a Heatmap using Plotly
    fig = px.imshow(pd_df,
                    labels=dict(x="Long Window", y="Short Window", color="Final Asset Value"),
                    x=pd_df.columns,
                    y=pd_df.index,
                    title='Heatmap of Results'
                   )

    fig.update_xaxes(side="top")  # This moves the x-axis labels to the top of the heatmap for better readability
    fig.show()


In [53]:
db_path = r'E:\git_repos\stock_analysis\data\raw_stock_price\stock_bar_data.db'
conn = duckdb.connect(db_path)
query = """
SELECT * FROM NVDA_hour_1
"""
stock_df = conn.execute(query).pl().sort('timestamp')
conn.close()

In [54]:
short_range = range(1, 20)
long_range = range(14, 100)

results_df = sensitivity_analysis(stock_df, short_range, long_range)


: 

In [40]:
analyze_and_visualize(results_df)

Highest Value: 153308.29 at (16, '15')
Mean Value: 105228.45401468789
Median Value: 104744.88500000001
Percentage Difference: 45.69%


In [31]:
test_strategy((15, 15), stock_df)

Cash,Shares Held,Asset Value,Buy Point,Sell Point,Normalized Stock Price
i64,i64,f64,bool,bool,f64
100000,0,100000.0,null,null,100000.0
100000,0,100000.0,null,null,99121.846439
100000,0,100000.0,null,null,95842.985301
100000,0,100000.0,null,null,93292.759971
100000,0,100000.0,null,null,96177.820831
100000,0,100000.0,null,null,95699.7852
100000,0,100000.0,null,null,96643.22116
100000,0,100000.0,null,null,98433.222423
100000,0,100000.0,null,null,97921.49265
